In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, TensorDataset
import random
import numpy as np
from sklearn.decomposition import PCA
from typing import Tuple
from tqdm import tqdm
from scipy.spatial import distance_matrix
import pandas as pd

In [2]:
root = "/Users/kristiansjorslevnielsen/Documents/DVP7/"
# load data
X_train = torch.tensor( np.load(root + "Features/cifar10/train_features_vgg16_cifar10.npy" ) )
y_train = np.load(root + "Features/cifar10/train_labels_vgg16_cifar10.npy" )

X_test = torch.tensor( np.load(root + "Features/cifar10/test_features_vgg16_cifar10.npy" ) )
y_test = np.load(root + "Features/cifar10/test_labels_vgg16_cifar10.npy")

y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [118]:
"""
root = "/Users/kristiansjorslevnielsen/Documents/DVP7/"
# load data
X_train = torch.tensor( np.load(root + "Features/NusWidePCA/X_train_nus_wide.npy" ) )
y_train = np.load(root + "Features/NusWidePCA/y_train_nus_wide.npy" )

X_test = torch.tensor( np.load(root + "Features/NusWidePCA/X_test_nus_wide.npy" ) )
y_test = np.load(root + "Features/NusWidePCA/y_test_nus_wide.npy")

y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
"""

'\nroot = "/Users/kristiansjorslevnielsen/Documents/DVP7/"\n# load data\nX_train = torch.tensor( np.load(root + "Features/NusWidePCA/X_train_nus_wide.npy" ) )\ny_train = np.load(root + "Features/NusWidePCA/y_train_nus_wide.npy" )\n\nX_test = torch.tensor( np.load(root + "Features/NusWidePCA/X_test_nus_wide.npy" ) )\ny_test = np.load(root + "Features/NusWidePCA/y_test_nus_wide.npy")\n\ny_train_tensor = torch.tensor(y_train, dtype=torch.long)\ny_test_tensor = torch.tensor(y_test, dtype=torch.long)\n'

In [119]:
#pca = PCA(n_components=128)
#pca.fit(X_train)
#X_train = torch.tensor(pca.transform(X_train), dtype=torch.float)
#X_test = torch.tensor(pca.transform(X_test), dtype=torch.float)

In [120]:
def meanAveragePrecision1(test_hashes, training_hashes, test_labels, training_labels):
    aps = []
    num_queries = len(test_hashes)
    for i in tqdm(range(num_queries)):
        label = test_labels[i]
        distances = (training_hashes != test_hashes[i]).sum(axis=1)  # Hamming distance
        tp = (training_labels == label)  # True positive indicator
        hash_df = pd.DataFrame({"distances": distances, "tp": tp.cpu().numpy()})
        hash_df = hash_df.sort_values(by="distances").reset_index(drop=True)
        hash_df["tp_cumsum"] = hash_df["tp"].cumsum()
        hash_df["precision"] = hash_df["tp_cumsum"] / (np.arange(len(hash_df)) + 1)
        ap = hash_df["precision"].where(hash_df["tp"] == 1).mean() if hash_df["tp"].sum() > 0 else 0
        aps.append(ap)

    return np.mean(aps)


In [121]:
def one_hot_encode(a):
    b = np.zeros((a.size, a.max() + 1))
    b[np.arange(a.size), a] = 1
    return b

In [122]:
def meanAveragePrecision(test_hashes, training_hashes, test_labels, training_labels):
    aps = []
    if len(training_labels.shape) == 1:
        training_labels = one_hot_encode(training_labels)
        test_labels = one_hot_encode(test_labels)
    for i, test_hash in enumerate(tqdm(test_hashes)):
        label = test_labels[i]
        distances = np.abs(training_hashes - test_hashes[i]).sum(axis=1)
        tp = np.where((training_labels*label).sum(axis=1)>0, 1, 0)
        hash_df = pd.DataFrame({"distances":distances, "tp":tp}).reset_index()
        hash_df = hash_df.sort_values(["distances", "index"]).reset_index(drop=True)
        hash_df = hash_df.drop(["index", "distances"], axis=1).reset_index()
        hash_df = hash_df[hash_df["tp"]==1]
        hash_df["tp"] = hash_df["tp"].cumsum()
        hash_df["index"] = hash_df["index"] +1 
        precision = np.array(hash_df["tp"]) / np.array(hash_df["index"])
        ap = precision.mean()
        aps.append(ap)
    
    return np.array(aps).mean()

In [123]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SUBIC_encoder(nn.Module): 
    def __init__(self, input_size=4096, bits=48, num_classes=10, num_blocks=6, block_size=8):
        super(SUBIC_encoder, self).__init__()
       
        assert bits % num_blocks == 0, "Bits must be divisible by num_blocks"

        self.input_size = input_size
        self.bits = bits 
        self.num_blocks = num_blocks
        self.block_size = block_size
        

        # Define the encoder structure
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 256), 
            nn.ReLU(),
            nn.Linear(256, bits)
        )  # Outputs binary feature vectors
        
        self.fc3 = nn.Linear(bits, num_classes)  # Logits for num_classes
    
    def block_softmax(self, x):
        
        batch_size = x.shape[0]
        block_size = x.shape[1] // self.num_blocks
        
        # Ensure that x has the expected shape
        assert x.shape[1] == self.bits, f"Expected shape [batch_size, {self.bits}], got {x.shape}"
        
        # Reshape and apply softmax
        x = x.view(batch_size, self.num_blocks, block_size)
        x = F.softmax(x, dim=-1) 
        return x.view(batch_size, -1) #-1 refers to the value that will match the original elements 
    
    def block_one_hot(self, x):
        batch_size = x.shape[0]

        x = x.view(batch_size, self.num_blocks, self.block_size)
        max_indices = x.argmax(dim=-1, keepdim=True)
        
        # Create one-hot encoding
        one_hot = torch.zeros_like(x).scatter_(-1, max_indices, 1)

        return one_hot.view(batch_size, self.bits)
    
    def forward(self, x, use_one_hot=False):
        # Ensure x is a flat tensor before passing to encoder
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)  # Flatten if necessary

        z = self.encoder(x)

        if use_one_hot:
            binary_codes = self.block_one_hot(z)
        else:
            binary_codes = self.block_softmax(z)

        class_probs = F.softmax(self.fc3(binary_codes), dim=-1) 

        return class_probs, binary_codes


In [124]:
def entropy(p):
    entropy_result = -torch.sum(p * torch.log2(p + 1e-30), dim=-1)
    return entropy_result

def cross_entropy(class_prob, target):
    if len(target.shape) >1:
        s = (class_prob*target).sum(axis=1)
    else:
        s = class_prob[torch.arange(len(target)), target]
    return -torch.log2(s)/torch.log2(torch.FloatTensor([class_prob.shape[1]]))

def compute_total_loss(class_probs, target, binary_codes, num_blocks, block_size, gamma=0.5, mu=0.5):
    """
    Computes the total loss, which includes:
    - Cross-entropy classification loss
    - Mean entropy loss (encouraging one-hot encoding within each block)
    - Batch entropy loss (encouraging uniform distribution across blocks)
    
    Parameters:
    - class_probs: The class probabilities from the classification layer.
    - target: The true labels.
    - binary_codes: The binary codes generated by the encoder.
    - num_blocks: The number of blocks in the binary codes.
    - block_size: The size of each block in the binary codes.
    - gamma: Weight for the mean entropy loss.
    - mu: Weight for the batch entropy loss.
    
    """



    classification_loss = cross_entropy(class_probs, target)

    batch_size = binary_codes.shape[0]
    binary_codes = binary_codes.view(batch_size, num_blocks, block_size) #used in structure encoding



    #Mean Entropy Loss (encourages each block to resemble a one-hot vector) using softmax binary code
    mean_entropy_loss = entropy(binary_codes).mean(dim=1)

    #Batch Entropy Loss (encourages uniform distribution across blocks)
    average_support = binary_codes.mean(dim=0)  
    batch_entropy_loss = entropy(average_support).mean(dim=0)

    #Combine losses with weights gamma and mu
    entropy_loss = (gamma * mean_entropy_loss - mu * batch_entropy_loss)/torch.log2(torch.FloatTensor([block_size]))
    total_loss = (classification_loss + entropy_loss).mean()
    
    
    return total_loss

#logits, binary_codes = model(X_train, use_one_hot=False)
#loss = compute_total_loss(logits, y_train_tensor, binary_codes, num_blocks=8, block_size=4, gamma=0.5, mu=0.05)

In [125]:
model = SUBIC_encoder(bits = 48, input_size=X_train.shape[1], num_classes = y_train.shape[-1], num_blocks = 16, block_size = 3)
optimizer = optim.Adam(model.parameters(), lr=0.001)
class_probs, binary_codes = model.forward(X_train, use_one_hot=False)

In [126]:
epochs = 100

train_dataset = TensorDataset(X_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = TensorDataset(X_test, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
losses = []

for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for images, labels in train_loader:
        
        images, labels = images.to(device), labels.to(device)
        class_probs, binary_codes = model.forward(images, use_one_hot=False)


        # Compute loss and update model
        loss = compute_total_loss(class_probs, labels, binary_codes, num_blocks=16, block_size=3, gamma=0.5, mu=0.5)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    losses.append(total_loss / len(train_loader))
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader):.4f}")


Epoch [1/100], Loss: -0.1481
Epoch [2/100], Loss: -0.4006
Epoch [3/100], Loss: -0.4158
Epoch [4/100], Loss: -0.4215
Epoch [5/100], Loss: -0.4245
Epoch [6/100], Loss: -0.4259
Epoch [7/100], Loss: -0.4269
Epoch [8/100], Loss: -0.4280
Epoch [9/100], Loss: -0.4285
Epoch [10/100], Loss: -0.4301
Epoch [11/100], Loss: -0.4315
Epoch [12/100], Loss: -0.4317
Epoch [13/100], Loss: -0.4322
Epoch [14/100], Loss: -0.4326
Epoch [15/100], Loss: -0.4330
Epoch [16/100], Loss: -0.4329
Epoch [17/100], Loss: -0.4354
Epoch [18/100], Loss: -0.4347
Epoch [19/100], Loss: -0.4363
Epoch [20/100], Loss: -0.4350
Epoch [21/100], Loss: -0.4356
Epoch [22/100], Loss: -0.4345
Epoch [23/100], Loss: -0.4357
Epoch [24/100], Loss: -0.4361
Epoch [25/100], Loss: -0.4354
Epoch [26/100], Loss: -0.4355
Epoch [27/100], Loss: -0.4362
Epoch [28/100], Loss: -0.4366
Epoch [29/100], Loss: -0.4368
Epoch [30/100], Loss: -0.4376
Epoch [31/100], Loss: -0.4370
Epoch [32/100], Loss: -0.4378
Epoch [33/100], Loss: -0.4377
Epoch [34/100], Los

KeyboardInterrupt: 

In [ ]:
class_probs, hash = model.forward(X_test, use_one_hot=False)

In [ ]:
class_probs[0]

tensor([1.0351e-03, 1.4728e-03, 7.1268e-03,  ..., 1.5334e-10, 1.7459e-10,
        2.3274e-10], grad_fn=<SelectBackward0>)

In [ ]:
_, train_binary_codes = model(X_train, use_one_hot=True)
_, test_binary_codes = model(X_test, use_one_hot=True)

train_binary_codes = train_binary_codes.detach().numpy()
test_binary_codes = test_binary_codes.detach().numpy()
test_binary_codes[9000:,].shape

(1000, 48)

In [ ]:
_, train_binary_codes = model(X_train, use_one_hot=True)
_, test_binary_codes = model(X_test, use_one_hot=True)

train_binary_codes = train_binary_codes.detach().numpy()
test_binary_codes = test_binary_codes.detach().numpy()

database = train_binary_codes
database_label = y_train
query_set = test_binary_codes
query_set_label = y_test


meanAveragePrecision(query_set, database, query_set_label, database_label)

100%|██████████| 10000/10000 [02:03<00:00, 81.06it/s]


0.6837244289367512

In [ ]:
model.eval()
all_query_codes, all_query_labels = [], []
all_db_codes, all_db_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        _, binary_codes = model(images, use_one_hot=True)

        # Ensure binary_codes is a tensor
        if not isinstance(binary_codes, torch.Tensor):
            raise TypeError("Expected binary_codes to be a tensor.")
        
        all_db_codes.append(binary_codes)
        all_db_labels.append(labels)

        if len(all_query_codes) == 0:  
            all_query_codes.append(binary_codes.clone())  
            all_query_labels.append(labels.clone())

# Concatenate all tensors
all_query_codes = torch.cat(all_query_codes, dim=0)
all_query_labels = torch.cat(all_query_labels, dim=0)
all_db_codes = torch.cat(all_db_codes, dim=0)
all_db_labels = torch.cat(all_db_labels, dim=0)

# Calculate MAP Score
map_score = meanAveragePrecision(
    all_query_codes,
    all_db_codes,
    all_query_labels,
    all_db_labels
    )

print(f"MAP Score: {map_score:.5f}")

TypeError: 'builtin_function_or_method' object cannot be interpreted as an integer